In [ ]:
import pyodbc

# ==============================
# CONFIGURAÇÕES DE CONEXÃO
# ==============================
import pyodbc

server = 'DESKTOP-F0DI39E'
database = 'Ourobase_CadastroDocumentacaoBI'

conn_str = f'''
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
'''

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

print("✅ Conectado com sucesso via Windows Authentication")

# ==============================
# DROP TABELA (SE EXISTIR)
# ==============================
cursor.execute("""
IF OBJECT_ID('dbo.st_ProjetoBI', 'U') IS NOT NULL
    DROP TABLE dbo.st_ProjetoBI
""")
conn.commit()

# ==============================
# CREATE TABLE
# ==============================
cursor.execute("""
CREATE TABLE dbo.st_ProjetoBI (
    id_ProjetoBI INT IDENTITY(1,1) PRIMARY KEY,

    str_Cliente VARCHAR(150) ,
    str_Usuario VARCHAR(150),
    str_Email VARCHAR(150),

    bit_StatusAtivo BIT,
    str_DescrInativo VARCHAR(255),

    str_TipoAcessoServidor VARCHAR(100),
    str_NomeServidor VARCHAR(150),
    str_NomeDataBase VARCHAR(150),

    str_NomeProjetoBI VARCHAR(150),
    bit_PossuiAplicativo BIT,
    str_TipoLicenca VARCHAR(100),

    bit_PossuiRegraNegocio BIT,
    str_DescrRegraRLSBI VARCHAR(255),

    int_QtdeAtualizacoes INT,
    str_AtualizacaoAgendada VARCHAR(100),

    bit_PossuiBIPersonalizado BIT,
    str_DescrBIPersonalizado VARCHAR(255),

    bit_MelhoriasBI BIT,
    str_DescrMelhoriasBI VARCHAR(255),
    dta_DataMelhoria DATE,

    int_NroPendencia INT,
    dta_DataInicioPendencia DATE,
    dta_DataFimPendencia DATE,

    int_NroAtendimento INT,
    dta_DataInicioAtendimento DATE,
    dta_DataFimAtendimento DATE,

    str_VersionamentoProjeto VARCHAR(50),
    dta_DataCadastro DATETIME DEFAULT GETDATE()
)
""")

conn.commit()
cursor.close()
conn.close()

print("✅ Tabela st_ProjetoBI criada com sucesso!")


✅ Conectado com sucesso via Windows Authentication
✅ Tabela st_ProjetoBI criada com sucesso!


In [16]:
import pandas as pd
import pyodbc

# ==============================
# CONFIGURAÇÕES
# ==============================
server = 'DESKTOP-F0DI39E'
database = 'Ourobase_CadastroDocumentacaoBI'
arquivo_excel = r'dbo_Clientes.xlsx'
tabela = 'dbo.st_ProjetoBI'

# ==============================
# CONEXÃO SQL SERVER (Windows Auth)
# ==============================
conn_str = f'''
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
'''

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
conn.autocommit = False

print('✅ Conectado ao SQL Server')

# ==============================
# LER EXCEL
# ==============================
df = pd.read_excel(arquivo_excel)
display( df.head(5))
print(f'📄 {len(df)} registros carregados do Excel')

# ==============================
# TRUNCATE TABLE
# ==============================
cursor.execute(f'TRUNCATE TABLE {tabela}')
conn.commit()
print('🧹 Tabela truncada com sucesso')

# ==============================
# INSERT DINÂMICO
# ==============================
colunas = ', '.join(df.columns)
placeholders = ', '.join(['?' for _ in df.columns])

sql_insert = f'''
INSERT INTO {tabela} ({colunas})
VALUES ({placeholders})
'''

# Converter DataFrame para lista de tuplas
dados = [tuple(row) for row in df.itertuples(index=False)]

cursor.executemany(sql_insert, dados)
conn.commit()

print('🚀 Carga finalizada com sucesso')

cursor.close()
conn.close()


✅ Conectado ao SQL Server


,id_ProjetoBI,str_Cliente,str_Usuario,str_Email,bit_StatusAtivo,str_DescrInativo,str_TipoAcessoServidor,str_NomeServidor,str_NomeDataBase,str_NomeProjetoBI,...,str_DescrMelhoriasBI,dta_DataMelhoria,int_NroPendencia,dta_DataInicioPendencia,dta_DataFimPendencia,int_NroAtendimento,dta_DataInicioAtendimento,dta_DataFimAtendimento,str_VersionamentoProjeto,dta_DataCadastro
0,NaN,SINGULAR,THAIS,SINGULAR@SINGULAR.COM.BR,1,NaN,LOCAL,SRV-BI,TESTE,TESTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0.4,NaN


📄 1 registros carregados do Excel
🧹 Tabela truncada com sucesso


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]O fluxo de entrada do protocolo TDS RPC (chamada de procedimento remoto) está incorreto. Parâmetro 4 (''): o valor fornecido não é uma instância válida do tipo de dados float. Verifique se há valores inválidos nos dados de origem. Exemplo de valor inválido: dados do tipo numérico com escala superior à precisão. (8023) (SQLExecDirectW)")